In [2]:
from model import get_models_and_path
models = get_models_and_path()


In [7]:
resFreeze = models[-4]
print(resFreeze[0].__class__.__name__)
print(resFreeze[1])
res = models[-5]
print(res[0].__class__.__name__)
print(res[1])


In [14]:
target = "gene1.0.bias"

for name, param in resFreeze[0].named_parameters():
    if name != target:
        continue
    
    print(name, ": ", param.data)
    
for name, param in res[0].named_parameters():
    if name != target:
        continue
    #print(name)
    print(name, ": ", param.data)
    

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../Training_Data/p009/Preprocessed_STDataset/gene_data.csv")
print(df.head())

        LYZ   SLC51B     DEFA5      CD27    RUBCNL      FLT1      AREG  \
0 -0.162695 -0.97812 -1.112690 -0.402535 -0.156612 -0.685642 -0.790518   
1 -0.357453 -0.97812  0.469045 -0.625870 -0.333559 -0.685642 -0.790518   
2  0.290787 -0.97812  1.531921  0.229230 -0.000078 -0.685642 -0.790518   
3  0.290787 -0.97812 -0.134141 -1.242845 -0.776494 -0.685642 -0.790518   
4 -0.357453 -0.97812 -0.134141 -0.625870  0.140268 -0.685642 -0.790518   

       IL1A    COL4A1    PHLDA2  ...     CXCR2      NKG7     MCM3   SLC26A2  \
0  0.316883 -0.403293 -0.060785  ... -0.626653 -0.526299 -0.86564  0.144542   
1  0.316883 -1.249125 -1.084120  ... -0.626653 -0.526299 -0.86564 -1.032822   
2  1.964727 -0.403293  0.570007  ... -0.626653 -0.526299 -0.86564 -1.032822   
3  0.316883 -0.403293 -1.084120  ... -0.626653 -0.526299 -0.86564 -0.304439   
4 -0.848575 -1.249125 -1.084120  ... -0.626653 -0.526299 -0.86564 -1.032822   

      TIGIT     OLFM4      KLK1     AXIN2      MUC2            tile  
0 -0.41010

In [4]:
print(df["MKI67"])

0      -0.482148
1      -0.482148
2      -0.482148
3      -0.482148
4      -0.482148
          ...   
2915   -0.482148
2916   -0.482148
2917   -0.482148
2918    1.414618
2919   -0.482148
Name: MKI67, Length: 2920, dtype: float64


In [9]:
df = pd.read_csv("../Training_Data/p013/Preprocessed_STDataset/gene_data.csv")
print(df.head())
mk = df["MKI67"]
rb = df["RUBCNL"]

#RUBCNL

        LYZ    SLC51B     DEFA5      CD27    RUBCNL      FLT1      AREG  \
0  0.194354  1.062390 -0.464035  0.395991  0.297868 -1.152690 -1.324021   
1  1.622593  1.062390  0.160855  1.045367  1.550454 -0.004348  0.820927   
2  1.487137  0.318017  0.609875  0.604993  0.974872 -1.152690 -1.324021   
3  1.622593  0.318017 -0.464035  0.539584  0.813911 -0.004348 -1.324021   
4  1.487137 -0.263620 -1.499122  0.666778  0.165409 -1.152690 -0.581869   

       IL1A    COL4A1    PHLDA2  ...     CXCR2      NKG7      MCM3   SLC26A2  \
0  0.004729 -0.202566 -0.823470  ...  0.418946 -0.731669 -1.087606 -0.942118   
1  0.672551 -1.662169  0.524157  ...  1.169946 -0.731669  0.502669 -0.014073   
2 -1.101473 -1.129536  0.524157  ... -0.825034 -0.731669  0.502669 -0.014073   
3  0.004729 -0.746809  0.524157  ... -0.825034 -0.731669 -0.095982 -0.014073   
4 -1.101473 -0.447880  0.716235  ...  1.169946 -0.731669 -0.095982 -0.014073   

      TIGIT     OLFM4      KLK1     AXIN2      MUC2           tile  

In [10]:
print(rb.min())
print(mk.min())


-3.623173041692125
-0.6384363382434257


In [11]:
rb.value_counts()

RUBCNL
-0.975272    42
-0.820521    38
-0.681907    36
 0.018293    35
-1.150418    33
             ..
 2.121521     1
 1.690388     1
 1.796714     1
-3.623173     1
 2.004438     1
Name: count, Length: 70, dtype: int64

In [12]:
mk.value_counts()


MKI67
-0.638436    622
 0.916830    205
 1.855756     60
 2.530430     19
 3.057383      7
 3.489823      3
 3.856554      3
 4.708216      1
Name: count, dtype: int64